In [1]:
import os
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

np.random.seed(0)

In [2]:
# #############################################################################
# # This method of data loading is not suitable for static graph in TF1.14, 
# # you can run this block in TF1.14 eager mode to use iterable objects.
# # If one day Rosetta support TF dynamic graph, these lines count.


# train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
# train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url), origin=train_dataset_url)
# print("Local copy of the dataset file: {}".format(train_dataset_fp))

# test_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
# test_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(test_dataset_url), origin=test_dataset_url)
# print("Local copy of the dataset file: {}".format(test_dataset_fp))

# column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
# feature_names = column_names[:-1]
# label_name = column_names[-1]
# print("Features: {}".format(feature_names))
# print("Label: {}".format(label_name))
# class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']

# batch_size = 32
# train_dataset = tf.data.experimental.make_csv_dataset(
#     train_dataset_fp,
#     batch_size,
#     column_names=column_names,
#     label_name=label_name,
#     num_epochs=1)
# print(train_dataset)
# test_dataset = tf.data.experimental.make_csv_dataset(
#     test_dataset_fp,
#     batch_size,
#     column_names=column_names,
#     label_name=label_name,
#     num_epochs=1,
#     shuffle=False)

# def pack_features_vector(features, labels):
#     """Pack the features into a single array."""
#     features = tf.stack(list(features.values()), axis=1)
#     return features, labels

# train_dataset = train_dataset.map(pack_features_vector)
# test_dataset = test_dataset.map(pack_features_vector)

# iter_train_dataset = iter(train_dataset)
# iter_test_dataset = iter(test_dataset)

# train_features, train_labels = next(iter_train_dataset)
# test_features, test_labels = next(iter_test_dataset)

# print(train_features, train_labels)
# print(test_features, test_labels)


In [3]:
###
# This block is to split the data into features and label cols, like tutorials, 
# you can copy this block into a new script, don't forget to import tensorflow, numpy and pandas

train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url), origin=train_dataset_url)
print("Local copy of the dataset file: {}".format(train_dataset_fp))

test_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
test_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(test_dataset_url), origin=test_dataset_url)
print("Local copy of the dataset file: {}".format(test_dataset_fp))

!pwd
# Copy Iris dataset into Rosetta dir for different party(ALL, PO or P1)
# !cp /home/username/.keras/datasets/iris_training.csv /path_to_Rosetta/example/tutorials/dsets/ALL/
# !cp /home/username/.keras/datasets/iris_test.csv /path_to_Rosetta/example/tutorials/dsets/ALL/

tutorials_path = os.path.abspath(os.path.join(os.getcwd(), "../tutorials/"))
print(tutorials_path)
print(os.getcwd())

iris_training_features = pd.read_csv(tutorials_path + "/dsets/ALL/iris_training.csv", \
    skiprows=[0], usecols=[0, 1, 2, 3])
iris_training_label = pd.read_csv(tutorials_path + "/dsets/ALL/iris_training.csv", \
    skiprows=[0], usecols=[4])
iris_test_features = pd.read_csv(tutorials_path + "/dsets/ALL/iris_test.csv", \
    skiprows=[0], usecols=[0, 1, 2, 3])
iris_test_label = pd.read_csv(tutorials_path + "/dsets/ALL/iris_test.csv", \
    skiprows=[0], usecols=[4])
# print(iris_training_features)
# print(iris_training_label)
# print(iris_test_features)
# print(iris_test_label)

iris_training_features.to_csv(tutorials_path + "/dsets/ALL/iris_training_features.csv", index=0)
iris_training_label.to_csv(tutorials_path + "/dsets/ALL/iris_training_label.csv", index=0)
iris_test_features.to_csv(tutorials_path + "/dsets/ALL/iris_test_features.csv", index=0)
iris_test_label.to_csv(tutorials_path + "/dsets/ALL/iris_test_label.csv", index=0)



# # For P0 and P1 in RTT version, they should do some preprocess on label.
# iris_training_label_P = pd.read_csv(tutorials_path + "/dsets/P1/iris_training_label.csv", header=None).to_numpy() - 1
# iris_test_label_P = pd.read_csv(tutorials_path + "/dsets/P1/iris_test_label.csv", header=None).to_numpy() - 1
# 
# print(iris_training_label_P, iris_training_label_P.shape)
# print(iris_test_label_P, iris_test_label_P.shape)
# 
# iris_training_label_P = pd.DataFrame(iris_training_label_P)
# iris_test_label_P = pd.DataFrame(iris_test_label_P)
# 
# print(iris_training_label_P)
# print(iris_test_label_P)
# 
# iris_training_label_P.to_csv(tutorials_path + "/dsets/P1/iris_training_label.csv", index=False, header=False)
# iris_test_label_P.to_csv(tutorials_path + "/dsets/P1/iris_test_label.csv", index=False, header=False)


Local copy of the dataset file: /home/juzix/.keras/datasets/iris_training.csv
Local copy of the dataset file: /home/juzix/.keras/datasets/iris_test.csv
/home/juzix/prRosetta/Rosetta/example/contrib
/home/juzix/prRosetta/Rosetta/example/tutorials
/home/juzix/prRosetta/Rosetta/example/contrib


In [4]:
EPOCHES = 1
BATCH_SIZE = 32
learning_rate = 0.003

# real data
# ######################################## difference from rosettta
tutorials_path = os.path.abspath(os.path.join(os.getcwd(), "../tutorials/"))
training_features_file_path = tutorials_path + "/dsets/ALL/iris_training_features.csv"
training_label_file_path = tutorials_path + "/dsets/ALL/iris_training_label.csv"
training_features = pd.read_csv(training_features_file_path, header=None).to_numpy()
training_label = pd.read_csv(training_label_file_path, header=None).to_numpy() - 1
test_features_file_path = tutorials_path + "/dsets/ALL/iris_test_features.csv"
test_label_file_path = tutorials_path + "/dsets/ALL/iris_test_label.csv"
test_features = pd.read_csv(test_features_file_path, header=None).to_numpy()
test_label = pd.read_csv(test_label_file_path, header=None).to_numpy() - 1
# ######################################## difference from rosettta
DIM_NUM = training_features.shape[1]

X = tf.placeholder(tf.float32, [None, DIM_NUM])
Y = tf.placeholder(tf.float32, [None, 1])
print(X)
print(Y)

# initialize W & b
W = tf.Variable(tf.zeros([DIM_NUM, 1]))
b = tf.Variable(tf.zeros([1]))
print(W)
print(b)

# predict
pred_Y = tf.matmul(X, W) + b
print(pred_Y)

# loss
loss = tf.square(Y - pred_Y)
loss = tf.reduce_mean(loss)
print(loss)

# optimizer
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
print(train)

init = tf.global_variables_initializer()
print(init)

with tf.Session() as sess:
    sess.run(init)
    xW, xb = sess.run([W, b])
    print("init weight:{} \nbias:{}".format(xW, xb))

    # train
    BATCHES = math.ceil(len(training_features) / BATCH_SIZE)
    for e in range(EPOCHES):
        for i in range(BATCHES):
            bX = training_features[(i * BATCH_SIZE): (i + 1) * BATCH_SIZE]
            bY = training_label[(i * BATCH_SIZE): (i + 1) * BATCH_SIZE]
            sess.run(train, feed_dict={X: bX, Y: bY})

            j = e * BATCHES + i
            if j % 50 == 0 or (j == EPOCHES * BATCHES - 1 and j % 50 != 0):
                xW, xb = sess.run([W, b])
                print("I,E,B:{:0>4d},{:0>4d},{:0>4d} weight:{} \nbias:{}".format(
                    j, e, i, xW, xb))

    # predict
    Y_pred = sess.run(pred_Y, feed_dict={X: test_features, Y: test_label})
#     print("Y_pred:\n", Y_pred)
#     print("Y_true:\n", test_label)
    Y_comp = np.hstack((Y_pred,test_label))
    print("Y_comp:\n", Y_comp)
    Y_pred_cls0 = np.where(Y_pred >= 0, Y_pred, -1)
    Y_pred_cls0 = np.where(Y_pred < 0, Y_pred_cls0, 0)
    Y_pred_cls2 = np.where(Y_pred < 0.05, Y_pred, 1)
    Y_pred_cls2 = np.where(Y_pred >= 0.05, Y_pred_cls2, 0)
    Y_pred_cls012 = Y_pred_cls0 + Y_pred_cls2
    Y_comp = np.hstack((Y_pred_cls012, test_label))
    print(Y_comp)
    acc = len([i for i in Y_pred_cls012 if i in test_label]) / len(test_label)
    print('acc: ', acc)

Tensor("Placeholder:0", shape=(?, 4), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 1), dtype=float32)
<tf.Variable 'Variable:0' shape=(4, 1) dtype=float32_ref>
<tf.Variable 'Variable_1:0' shape=(1,) dtype=float32_ref>
Tensor("add:0", shape=(?, 1), dtype=float32)
Tensor("Mean:0", shape=(), dtype=float32)
name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_Variable/ApplyGradientDescent"
input: "^GradientDescent/update_Variable_1/ApplyGradientDescent"

name: "init"
op: "NoOp"
input: "^Variable/Assign"
input: "^Variable_1/Assign"

init weight:[[0.]
 [0.]
 [0.]
 [0.]] 
bias:[0.]
I,E,B:0000,0000,0000 weight:[[ 0.004875  ]
 [-0.0001125 ]
 [ 0.0099    ]
 [ 0.00395625]] 
bias:[0.000375]
I,E,B:0003,0000,0003 weight:[[-0.00494406]
 [-0.01271045]
 [ 0.02192803]
 [ 0.01086732]] 
bias:[-0.00313594]
Y_comp:
 [[ 0.03796146  0.        ]
 [ 0.0645804   1.        ]
 [-0.02758382 -1.        ]
 [ 0.04004803  0.        ]
 [ 0.03973525  0.        ]
 [ 0.03776864  0.        ]
 [-0.0508394